In [7]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from dotenv import load_dotenv
from urllib.parse import quote 
import os

In [5]:
load_dotenv()
DBUSER = os.getenv('DBUSER')
DBPASS = os.getenv('DBPASS')
DBHOST = os.getenv('DBHOST')
DBPORT = os.getenv('DBPORT')
DBNAME = os.getenv('DBNAME')

# db_settings = {
#    'dbname': DBNAME,
#    'user': DBUSER,
#    'password': DBPASS,
#    'host': DBHOST,
#    'port': DBPORT,
    #'options': "-c statement_timeout=0",
    #'keepalives': 1,
    #'keepalives_idle': 30,  # Number of seconds of inactivity after which a keepalive message is sent
    #'keepalives_interval': 10,  # Number of seconds between keepalive messages when no response is received
    #'keepalives_count': 5  # Number of attempts before concluding the connection is dead
#}
# Function to establish connection to PostgreSQL
# def connect_db(settings):
#    conn = psycopg2.connect(**settings)
    # conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
#    return conn

# connect_db(db_settings)

In [6]:
PG_CONNECTION_STRING = 'postgresql://' + DBUSER + ':%s@' + DBHOST + ':' + DBPORT + '/' + DBNAME
engine = create_engine(PG_CONNECTION_STRING % quote(DBPASS), echo=True)

In [ ]:
data_dir = '../data_src/'
data_file = 'GLODAPv2.2023_Indian_Ocean.csv'
df = pd.read_csv(data_dir + data_file)
# Remove G2 prefix
df.columns = [col[2:] if col.startswith('G2') else col for col in df.columns]
# Replace missing values
df.replace(-9999, np.nan, inplace=True)


In [11]:
print(df)

            expocode  cruise  station  region  cast    year  month   day  \
0       09AR19910925    65.0      8.0    16.0   1.0  1991.0   10.0   8.0   
1       09AR19910925    65.0      8.0    16.0   1.0  1991.0   10.0   8.0   
2       09AR19910925    65.0      8.0    16.0   1.0  1991.0   10.0   8.0   
3       09AR19910925    65.0      8.0    16.0   1.0  1991.0   10.0   8.0   
4       09AR19910925    65.0      8.0    16.0   1.0  1991.0   10.0   8.0   
...              ...     ...      ...     ...   ...     ...    ...   ...   
167743  49NZ20191205  4062.0     69.0    16.0   1.0  2019.0   12.0  22.0   
167744  49NZ20191205  4062.0     69.0    16.0   1.0  2019.0   12.0  22.0   
167745  49NZ20191205  4062.0     69.0    16.0   1.0  2019.0   12.0  22.0   
167746  49NZ20191205  4062.0     69.0    16.0   1.0  2019.0   12.0  22.0   
167747  49NZ20191205  4062.0     69.0    16.0   1.0  2019.0   12.0  22.0   

        hour  minute  ...  tocf  doc  docf  don  donf  tdn  tdnf  chla  chlaf  \
0     

In [12]:
print(df.columns.tolist())

['expocode', 'cruise', 'station', 'region', 'cast', 'year', 'month', 'day', 'hour', 'minute', 'latitude', 'longitude', 'bottomdepth', 'maxsampdepth', 'bottle', 'pressure', 'depth', 'temperature', 'theta', 'salinity', 'salinityf', 'salinityqc', 'sigma0', 'sigma1', 'sigma2', 'sigma3', 'sigma4', 'gamma', 'oxygen', 'oxygenf', 'oxygenqc', 'aou', 'aouf', 'nitrate', 'nitratef', 'nitrateqc', 'nitrite', 'nitritef', 'silicate', 'silicatef', 'silicateqc', 'phosphate', 'phosphatef', 'phosphateqc', 'tco2', 'tco2f', 'tco2qc', 'talk', 'talkf', 'talkqc', 'fco2', 'fco2f', 'fco2temp', 'phts25p0', 'phts25p0f', 'phtsinsitutp', 'phtsinsitutpf', 'phtsqc', 'cfc11', 'pcfc11', 'cfc11f', 'cfc11qc', 'cfc12', 'pcfc12', 'cfc12f', 'cfc12qc', 'cfc113', 'pcfc113', 'cfc113f', 'cfc113qc', 'ccl4', 'pccl4', 'ccl4f', 'ccl4qc', 'sf6', 'psf6', 'sf6f', 'sf6qc', 'c13', 'c13f', 'c13qc', 'c14', 'c14f', 'c14err', 'h3', 'h3f', 'h3err', 'he3', 'he3f', 'he3err', 'he', 'hef', 'heerr', 'neon', 'neonf', 'neonerr', 'o18', 'o18f', 'toc'

In [ ]:
    
# Write to PostgreSQL
# df.to_sql('glodap_observations', engine, if_exists='append', index=False)